#  Jogo BreakThrough

## Introdução à Inteligência Artificial edição 2022/23
### Projeto para avaliação

<img src=".\images\Picture0.png" alt="Drawing" style="width: 200px;"/>

### Grupo: 40

#### Elementos do Grupo

Nome: Francisco Correia

Número: 54685

Nome: Francisco Maia

Número: 55855

Nome: Alexandre Fonseca

Número: 55955

## Introdução
Este é um esqueleto do relatório que podem naturalmente expandir, colocando mais células de texto e/ou de código.

## Formulação do Jogo BreakThrough em termos de estados e de operadores

### Descrição da representação dos estados do jogo

In [165]:
from time import time
from jogos import *
from jogar import *

WHITE = 1
BLACK = 2

class EstadoBT_40:
    def __init__(self, to_move, utility, board, whites, blacks, moves=None):
        self.to_move = to_move
        self.utility = utility
        self.board = board
        self.whites = whites
        self.blacks = blacks
        self.moves = moves

    def __str__(self):
        player_chars = [".", "W", "B"]  # 0, WHITES and BLACKS
        board_str = ["-----------------"]
        for i in range(len(self.board), 0, -1):
            board_str.append(
                f"{i}|" + " ".join(map(lambda x: player_chars[x], self.board[i - 1]))
            )
        board_str.append("-+---------------")
        board_str.append(" |a b c d e f g h")
        return "\n".join(board_str)


class JogoBT_40(Game):
    def __init__(self, n=8):
        self.n = n
        whites = set((row, col) for row in range(2) for col in range(n))
        blacks = set((row, col) for row in range(n - 2, n) for col in range(n))
        board = [[0 for _ in range(n)] for _ in range(n)]
        for x, y in whites:
            board[x][y] = WHITE
        for x, y in blacks:
            board[x][y] = BLACK
        to_move = WHITE
        self.action_dict = self.compute_action_dict(n)
        self.initial = EstadoBT_40(to_move, 0, board, whites, blacks)

    def actions(self, state: EstadoBT_40):
        if state.moves:
            return state.moves
        moves = set()
        if state.to_move == WHITE:
            pieces, pieces_opponent = state.whites, state.blacks
        else:
            pieces, pieces_opponent = state.blacks, state.whites
        for pos in pieces:
            for target, move in self.action_dict[pos][state.to_move].items():
                # can't eat opponent's piece if it is directly in front of ours
                if target[1] == pos[1] and target in pieces_opponent:
                    continue
                if target not in pieces:  # don't eat our own pieces
                    moves.add(move)
        state.moves = sorted(moves)
        return state.moves

    def display(self, state):
        print(state)
        if not self.terminal_test(state):
            print(f'--NEXT PLAYER: {"W" if state.to_move == WHITE else "B"}')

    def executa(self, state: EstadoBT_40, valid_actions: "list[str]"):
        """Executa várias jogadas sobre um estado dado.
        Devolve o estado final."""
        result = state
        for move in valid_actions:
            result = self.result(result, move)
        return result

    def result(self, state: EstadoBT_40, move):
        board = [row[:] for row in state.board]  # deepcopy
        (old_row, old_col), (new_row, new_col) = self.convert_move(move)
        board[new_row][new_col] = state.to_move
        board[old_row][old_col] = 0
        whites, blacks = state.whites.copy(), state.blacks.copy()

        if state.to_move == WHITE:
            whites.remove((old_row, old_col))
            whites.add((new_row, new_col))
            blacks.discard((new_row, new_col))
            to_move = BLACK
        else:
            blacks.remove((old_row, old_col))
            blacks.add((new_row, new_col))
            whites.discard((new_row, new_col))
            to_move = WHITE

        return EstadoBT_40(
            to_move,
            self.compute_utility(new_row, state.to_move),
            board,
            whites,
            blacks,
        )

    def terminal_test(self, state: EstadoBT_40):
        return state.utility != 0 or len(self.actions(state)) == 0

    def utility(self, state: EstadoBT_40, player):
        # W: 1, B: -1
        return state.utility if player == WHITE else -state.utility

    def compute_utility(self, row, player):
        if 0 < row < self.n - 1:
            return 0
        return 1 if player == WHITE else -1

    def compute_action_dict(self, n):
        """
        Gera um dicionário de ações que associa pares de coordenadas
        ((x1, y1), (x2, y2)) para ações no formato dado no enunciado.
        Exemplo: ((0, 0), (1, 1)) -> a1-b2
        """

        def in_board(x):
            return 0 <= x < n

        ret = {}
        a_ord = ord("a")
        for row in range(n):
            for col in range(n):
                black_moves = {}
                white_moves = {}
                for i in filter(in_board, (row - 1, row + 1)):
                    # white pieces move upwards, black pieces downwards
                    moves = white_moves if i > row else black_moves
                    for j in filter(in_board, (col - 1, col, col + 1)):
                        moves[(i, j)] = "-".join(
                            [f"{chr(col + a_ord)}{row + 1}", f"{chr(j + a_ord)}{i + 1}"]
                        )
                # use None to pad tuple so we can use to_move as index
                ret[(row, col)] = (None, white_moves, black_moves)
        return ret

    # ? Usar dict inverso para mais desempenho
    def convert_move(self, move):
        """Converte uma ação no formato do enunciado
        para uma ação representada no estado interno.
        Exemplo: "a1-b2" -> ((0, 0), (1, 1))"""
        ord_a = ord("a")
        (old_col, old_row), (new_col, new_row) = move.split("-")
        return (
            (int(old_row) - 1, ord(old_col) - ord_a),
            (int(new_row) - 1, ord(new_col) - ord_a),
        )

### Testes da formulação

#### Situações iniciais dos jogos
Uso do construtor e "display" de jogos iniciais

Construção de um novo jogo com a situação inicial seguinte:

<img src=".\images\Picture1.png" alt="Drawing" style="width: 150px;"/>
<p style="text-align: center;">Figura 1</p>

### Método Display()
Tendo em conta o que foi explicado acima, a construção de um novo jogo tem como objetivo definir quais as posições iniciais das peças brancas e pretas, as brancas vão preencher as primeiras duas linhas e as pretas as últimas duas, o primeiro jogador a jogar, que será sempre o que joga com as peças brancas, e criar o dicionário de jogadas, que vai ser util para traduzir um par de cordenadas em jogadas do estilo "a1-a2".

A função compute_action_dict() devolve um dicionário que associa uma posição `(x, y)` usada no estado interno às ações disponíveis nessa posição.

As associações são feitas em função do jogador (`1` para peças brancas, `2` para peças pretas).

As ações são devolvidas num dicionário em que as chaves são a posição do destino e os valores são a ação em si.

In [166]:
j1 = JogoBT_40()

Eis o display desse estado inicial do jogo:

In [167]:
j1.display(j1.initial)

-----------------
8|B B B B B B B B
7|B B B B B B B B
6|. . . . . . . .
5|. . . . . . . .
4|. . . . . . . .
3|. . . . . . . .
2|W W W W W W W W
1|W W W W W W W W
-+---------------
 |a b c d e f g h
--NEXT PLAYER: W


### Método actions()

Podemos verificar quais as ações possiveís para o próximo jogador, neste caso 'W', a partir do método actions. Este método foi definido para iterar sobre todas as peças do jogador a jogar e de seguida iterar pelo dicionário de ações.
O dicionário está definido da seguinte forma:

```Python
(1,1) -> (
    None,
    1 (Whites): {(2,0): b2-a3, (2,1): b2-b3,(2,2): b2-c3},
    2 (Blacks): {(0,0): b2-a1, (0,1): b2-b1,(0,2): b2-c1}
)
```
Isto é, para a posição (1,1) uma peça branca pode mover-se para (2,0), (2,1), (2,2) e uma peça preta para (0,0), (0,1), (0,2).

Durante a iteração pelas jogadas que podem ser executadas verificamos:
- Se na posição "à frente" da posição inicial da peça existe uma peça do adversário
- Se não existem peças do jogador na posição para as posições que a peça se pode mover

A jogada só é válida se **ambas** as condições forem falsas.

In [168]:
to_move = j1.initial.to_move
player = 'W' if to_move == 1 else 'B'
print("Next Player:", player)
print("Ações possíveis:", j1.actions(j1.initial))

Next Player: W
Ações possíveis: ['a2-a3', 'a2-b3', 'b2-a3', 'b2-b3', 'b2-c3', 'c2-b3', 'c2-c3', 'c2-d3', 'd2-c3', 'd2-d3', 'd2-e3', 'e2-d3', 'e2-e3', 'e2-f3', 'f2-e3', 'f2-f3', 'f2-g3', 'g2-f3', 'g2-g3', 'g2-h3', 'h2-g3', 'h2-h3']


## Jogos entre jogadores simples
Nesta secção irão realizar alguns jogos, para verificar a modelização

Jogo entre dois jogadores random:

In [169]:
j2 = JogoBT_40()
player1 = Jogador("Randall", random_player)
player2 = Jogador("Randy", random_player)
game_to_display = joga11(j2, player1, player2)
mostraJogo(j2, game_to_display, False)

Randall vs Randy
Ganham as Blacks


Jogo entre um jogador random e um jogador alfabeta, com depth 2, que utiliza como eval_fun a função utility:

In [170]:
player3 = JogadorAlfaBeta("Alfaiate", 2, j2.utility)
mostraJogo(j2, joga11(j2, player3, player1))

Alfaiate vs Randall
Ganham as Whites


Campeonato entre 4 jogadores, dois random's e dois alfabeta, com depth 2, que utilizam a função utility como eval_fun:

In [171]:
player4 = JogadorAlfaBeta("Alberta", 2, j2.utility)
faz_campeonato(j2, [player1, player2, player3, player4], 10)

1 Randall Randy --vencedor= Randy
2 Randall Alfaiate --vencedor= Alfaiate
3 Randall Alberta --vencedor= Alberta
4 Randy Randall --vencedor= Randall
5 Randy Alfaiate --vencedor= Alfaiate
6 Randy Alberta --vencedor= Alberta
7 Alfaiate Randall --vencedor= Alfaiate
8 Alfaiate Randy --vencedor= Alfaiate
9 Alfaiate Alberta --vencedor= Alberta
10 Alberta Randall --vencedor= Alberta
11 Alberta Randy --vencedor= Alberta
12 Alberta Alfaiate --vencedor= Alfaiate
JOGADOR VITÓRIAS
Alfaiate       5
Alberta        5
Randall        1
Randy          1


Faça o display de um dos jogos realizados atrás

In [172]:
jogadores, jogadas, vencedor = game_to_display
print("Jogo entre", jogadores[0], "e", jogadores[1], "\n")
print("As jogadas efetuadas foram: ", jogadas, "\n")
print("Vencedor do jogo:", jogadores[0] if vencedor == 1 else jogadores[1])

Jogo entre Randall e Randy 

As jogadas efetuadas foram:  ['b2-b3', 'h7-g6', 'c2-c3', 'd7-d6', 'g2-h3', 'g6-f5', 'd2-d3', 'f5-f4', 'c3-b4', 'b7-a6', 'b4-c5', 'f7-f6', 'b1-b2', 'e8-d7', 'e1-d2', 'c7-b6', 'h3-g4', 'b6-b5', 'd3-e4', 'f4-f3', 'g4-f5', 'f8-f7', 'f2-g3', 'a8-b7', 'f5-g6', 'f7-e6', 'g6-h7', 'g8-f7', 'e2-e3', 'f3-g2', 'g3-f4', 'd6-e5', 'c5-c6', 'f7-g6', 'f1-f2', 'd7-c6', 'd2-c3', 'b5-c4', 'e4-f5', 'd8-c7', 'd1-c2', 'c6-d5', 'f4-g5', 'c4-b3', 'c3-c4', 'e5-e4', 'c4-c5', 'e6-f5', 'h2-g3', 'e4-d3', 'e3-e4', 'c7-c6', 'g1-h2', 'g2-f1'] 

Vencedor do jogo: Randy


## Exemplos de jogadores alfabeta
 Descreva e teste nesta secção as várias funções de avaliação desenvolvidas tanto para o ataque como para a defesa.

### Belarmino

Primeiramente, antes de definirmos as nossas próprias funções de avaliação, definimos o belarmino:

In [173]:
def f_aval_belarmino(estado: EstadoBT_40, jogador):
    res = 0
    n = len(estado.board)
    if jogador == WHITE:
        for row, _ in estado.whites:
            x = row + 1
            res += x**x
    else:
        for row, _ in estado.blacks:
            x = n - row
            res += x**x
    return res

### Funções de avaliação definidas por nós

Como último jogador, vamos chama-lo Heuracio, definimos um jogador baseado no anterior mas com uma nova função ```piece_value() ``` 

## Exemplos de jogos entre alguns desses jogadores e o Belarmino

## Exemplos de jogos entre dois dos vários jogadores desenvolvidos

## Processo de selecção dos jogadores para o torneio
Descreva o processo de selecção dos jogadores campeões, para entrar no campeonato "todos contra todos".



*...aqui* 

<span style="color:magenta"> É a parte mais importante do relatório, justificando porque convocaram o vosso Ronaldo para o campeonato. Se jogaram com vários jogadores (ou seja, várias funções de avaliação) e fizeram um torneio privado de selecção, podem apresentar aqui uma tabela com esses dados. </span>